# P07: Introdução ao Spark com Python


## MLlib: Estatística Descritiva

## Como obter o `SparkContext`

O `SparkContext` Carregado automaticamente quando o notebook é iniciado pelo PySpark

In [ ]:
# Somente necessário quando o notebook não é carregado pelo PySpark
#import pyspark
#sc = pyspark.SparkContext(appName="P7")

# Quando o notebook é carregado pelo PySpark, a variável sc é disponibilizada automaticamente
sc

## Obtendo o conjunto de dados de análise reduzido

Usaremos um conjunto reduzido de dados (10%) da Copa KDD de 1999, que contém quase meio milhão de registros. O arquivo é fornecido como um *Gzip*.

In [ ]:
import urllib.request as request
f = request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

Atenção! Lembre-se de colocar o arquivo baixado no `HDFS` (caso ainda não tenha feito). Além disso, inicie o `HDFS` e o `Yarn`.

```bash
start-dfs.sh
start-yarn.sh
hdfs dfs -put kddcup.data_10_percent.gz /usr/hduser
```

In [ ]:
nome_arquivo = "./kddcup.data_10_percent.gz"
rdd = sc.textFile(nome_arquivo)

In [2]:
from pyspark.mllib.stat import Statistics 
from math import sqrt 

In [ ]:
# Obter estatísticas de colunas texto não dá certo
resumo = Statistics.colStats(rdd) # Essa linha deve retornar erro

In [ ]:
# Criando uma função que filtra as colunas texto

import numpy as np

def colunas_numericas(linha):
    todas_colunas_dessa_linha = linha.split(",")
    indice_colunas_texto = [1,2,3,41]
    linha_numerica = [item for indice, item in enumerate(todas_colunas_dessa_linha) if indice not in indice_colunas_texto]
    return np.array([float(elemento) for elemento in linha_numerica])

rdd_numerico = rdd.map(colunas_numericas)

## Estatística Descritiva

In [ ]:
estatistica_obj = Statistics.colStats(rdd_numerico)

# Resultados somente para a coluna 0: Duração
print("Estatística de Duração:")
print(" Média: {}".format(round(estatistica_obj.mean()[0],3)))
print(" Desvio padrão: {}".format(round(sqrt(estatistica_obj.variance()[0]),3)))
print(" Valor máximo: {}".format(round(estatistica_obj.max()[0],3)))
print(" Valor mínimo: {}".format(round(estatistica_obj.min()[0],3)))
print(" Contagem do valor total: {}".format(estatistica_obj.count()))
print(" Número de valores não zero: {}".format(estatistica_obj.numNonzeros()[0]))